In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf

### Reading Data

In [2]:
path_to_file = '../DATA/shakespeare.txt'

In [3]:
text = open(path_to_file, 'r').read()
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [4]:
char_vocab = set(text)
char_map = {char:ind for ind, char in enumerate(char_vocab)}
ind_to_char = np.array(char_vocab)

encoded_text = np.array([char_map[c] for c in text])

In [5]:
seq_len = 120
total_seqs = len(text) // (seq_len+1)
total_seqs

45005

In [6]:
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

In [7]:
def create_seq_targets(seq):
    input_txt = seq[:-1]
    target_txt = seq[1:]
    return input_txt, target_txt

In [8]:
dataset = sequences.map(create_seq_targets)

In [9]:
batch_size = 128
buffer_size = 10000
dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)
dataset

<_BatchDataset element_spec=(TensorSpec(shape=(128, 120), dtype=tf.int64, name=None), TensorSpec(shape=(128, 120), dtype=tf.int64, name=None))>

In [10]:
vocab_size = len(char_vocab)
embed_dims = 64
rnn_neurons = 1024

In [11]:
from tensorflow.keras.losses import sparse_categorical_crossentropy
help(sparse_categorical_crossentropy)

Help on function sparse_categorical_crossentropy in module keras.src.losses.losses:

sparse_categorical_crossentropy(y_true, y_pred, from_logits=False, ignore_class=None, axis=-1)
    Computes the sparse categorical crossentropy loss.
    
    Args:
        y_true: Ground truth values.
        y_pred: The predicted values.
        from_logits: Whether `y_pred` is expected to be a logits tensor. By
            default, we assume that `y_pred` encodes a probability distribution.
        ignore_class: Optional integer. The ID of a class to be ignored during
            loss computation. This is useful, for example, in segmentation
            problems featuring a "void" class (commonly -1 or 255) in
            segmentation maps. By default (`ignore_class=None`), all classes are
            considered.
        axis: Defaults to `-1`. The dimension along which the entropy is
            computed.
    
    Returns:
        Sparse categorical crossentropy loss value.
    
    Examples:
    


In [12]:
# Our inputs are OHE so from_logits should be true
def sparse_cat_loss(y_true, y_pred):
    return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

In [18]:
def create_model(vocab_size, embed_dims, rnn_neurons, batch_size, seq_len):
    model = Sequential()
    # model.add(Embedding(vocab_size, embed_dims, input_shape=(batch_size, None)))
    model.add(Embedding(input_dim=vocab_size, output_dim=embed_dims, input_length=batch_size))
    model.add(GRU(rnn_neurons, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))
    model.add(Dense(vocab_size))

    model.compile(loss=sparse_cat_loss, optimizer='adam')
    return model

In [19]:
model = create_model(vocab_size, embed_dims, rnn_neurons, batch_size, seq_len)
model.summary()

/Users/arfsyed/anaconda3/envs/tf_experiments/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [20]:
model.fit(dataset, epochs=30)

Epoch 1/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 475s 1s/step - loss: 2.8707
Epoch 2/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 492s 1s/step - loss: 1.6452
Epoch 3/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 470s 1s/step - loss: 1.4031
Epoch 4/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 473s 1s/step - loss: 1.3079
Epoch 5/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 486s 1s/step - loss: 1.2558
Epoch 6/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 485s 1s/step - loss: 1.2223
Epoch 7/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 483s 1s/step - loss: 1.1947
Epoch 8/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 479s 1s/step - loss: 1.1740
Epoch 9/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 482s 1s/step - loss: 1.1541
Epoch 10/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 472s 1s/step - loss: 1.1387
Epoch 11/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 477s 1s/step - loss: 1.1238
Epoch 12/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 487s 1s/step - loss: 1.1110
Epoch 13/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 494s 1s/step - loss: 1.0976
Epoch 14/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 496s 1s/step - loss: 1.0869
Epoch 15/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 49

In [21]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (128, 120, 64)         │         5,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (128, 120, 1024)       │     3,348,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (128, 120, 84)         │        86,100 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,319,870 (39.37 MB)

 Trainable params: 3,439,956 (13.12 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,879,914 (26.24 MB)

In [23]:
pred = model.predict(np.array([[0]]))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


In [25]:
pred.shape

(128, 1, 84)